In [1]:
import json
import os

extracted_dir = "./data/documents/extracted_texts2"
alter2 = {}
error = []
for fname in os.listdir(extracted_dir):
    if fname.endswith(".json"):
        try:
            with open(os.path.join(extracted_dir, fname), "r", encoding="utf-8") as f:
                data = json.load(f)
                bill_num = data["title"].split("_")[0]
                if data.get("bill_number") == "" or data.get("title") == "":
                    raise ValueError(f"Missing required fields in {fname}")
                if not bill_num.endswith(data["bill_number"]):
                    raise ValueError(f"Bill number mismatch in {fname}")
                if data["is_alternative"]:
                    alter2[bill_num] = data["alternative_bill_numbers"]
                else:
                    if "(대안)" in data["title"]:
                        raise ValueError(f"Title contains '대안' but is not marked as alternative in {fname}")
        except Exception as e:
            print(f"Error processing {e}")
            error.append((str(e)))

In [37]:
import pandas as pd

bills = pd.read_json("./data/assembly/formatted/bills.json", orient="records")
bill_details = pd.read_json("./data/assembly/formatted/bill_details.json", orient="records")
proposers = pd.read_json("./data/assembly/formatted/proposer_bills.json", orient="records")
members = pd.read_json("./data/assembly/raw/cur_members.json", orient="records")

## 발의안

In [48]:
import pandas as pd

# 날짜 컬럼 리스트
date_cols = [
    "COMMITTEE_DT",
    "CMT_PRESENT_DT",
    "CMT_PROC_DT",
    "LAW_SUBMIT_DT",
    "LAW_PRESENT_DT",
    "LAW_PROC_DT",
    "PROC_DT",
]

# 날짜 컬럼을 datetime으로 변환 (null/None/NaN 안전하게)
for col in date_cols:
    bill_details[col] = pd.to_datetime(bill_details[col], errors="coerce")

# 기간 계산 (결과가 NaN이면 -1로 대체)
bill_details = bill_details[bill_details["PROC_RESULT"]!="철회"]
bill_details["CMT_MEET"] = (bill_details["CMT_PRESENT_DT"] - bill_details["COMMITTEE_DT"]).dt.days.fillna(-1).astype(int)
bill_details["CMT_PROC"] = (bill_details["CMT_PROC_DT"] - bill_details["CMT_PRESENT_DT"]).dt.days.fillna(-1).astype(int)
bill_details["LAW_"] = (bill_details["LAW_SUBMIT_DT"] - bill_details["CMT_PROC_DT"]).dt.days.fillna(-1).astype(int)
bill_details["LAW_MEET"] = (bill_details["LAW_PRESENT_DT"] - bill_details["LAW_SUBMIT_DT"]).dt.days.fillna(-1).astype(int)
bill_details["LAW_PROC"] = (bill_details["LAW_PROC_DT"] - bill_details["LAW_PRESENT_DT"]).dt.days.fillna(-1).astype(int)
bill_details["PROC"] = (bill_details["PROC_DT"] - bill_details["LAW_PROC_DT"]).dt.days.fillna(-1).astype(int)
bill_details["TOTAL_DT"] = (bill_details["PROC_DT"] - bill_details["COMMITTEE_DT"]).dt.days.fillna(-1).astype(int)

# 각 단계별 전환율 계산
total = len(bill_details)
cmt_meet = (bill_details["CMT_MEET"] != -1).sum()
cmt_proc = (bill_details["CMT_PROC"] != -1).sum()
law_meet = (bill_details["LAW_MEET"] != -1).sum()
law_proc = (bill_details["LAW_PROC"] != -1).sum()
proc = (bill_details["PROC"] != -1).sum()

print("위원회 회부 전환율:", round(cmt_meet / total, 3))
print("위원회 처리 전환율:", round(cmt_proc / total, 3))
print("본회의 회부 전환율:", round(law_meet / total, 3))
print("본회의 처리 전환율:", round(law_proc / total, 3))
print("공포/처리 전환율:", round(proc / total, 3))




bill_details[["CMT_MEET", "CMT_PROC", "LAW_", "LAW_MEET", "LAW_PROC", "PROC", "TOTAL_DT"]].replace(-1, pd.NA).mean(skipna=True)
# bill_details[bill_details["TOTAL_DT"] != -1][["CMT_MEET", "CMT_PROC", "LAW_MEET", "LAW_PROC", "PROC", "TOTAL_DT"]]


위원회 회부 전환율: 0.658
위원회 처리 전환율: 0.174
본회의 회부 전환율: 0.05
본회의 처리 전환율: 0.05
공포/처리 전환율: 0.05


CMT_MEET     71.931159
CMT_PROC     39.353973
LAW_           0.06203
LAW_MEET     17.526515
LAW_PROC      1.216475
PROC          2.919386
TOTAL_DT    126.598185
dtype: object

## 의원회별

In [ ]:
"\uc218\uc815\uac00\uacb0"

'수정가결'

In [ ]:
committe_stats = bills.groupby("COMMITTEE_NAME").agg(
    BILL_COUNT=("BILL_ID", "count"),
    PASS_COUNT=("BILL_ID", lambda x: bills.loc[x.index, "STATUS"].isin(["원안가결", "수정가결"]).sum()),
    NEW_BILL_COUNT=("BILL_ID", lambda x: bills.loc[x.index, "STATUS"].isin(["원안가결", "수정가결", "수정안반영폐기", "대안반영폐기"]).sum()),
).reset_index().sort_values("BILL_COUNT", ascending=False)

committe_stats["PASS_RATE"] = (
    committe_stats["PASS_COUNT"] / committe_stats["BILL_COUNT"]
).round(2) 

committe_stats["PROCESSED_RATE"] = (
    committe_stats["NEW_BILL_COUNT"] / committe_stats["BILL_COUNT"]
).round(2) 
committe_stats.sort_values("BILL_COUNT", ascending=False)

In [ ]:
committees = bills["COMMITTEE_NAME"].unique()
for committee in committees:
    if committee is None:
        continue
    print(f"## {committee}")
    print(
        bills[bills["COMMITTEE_NAME"] == committee]
        .groupby("BILL_NAME")
        .count()[["BILL_ID"]]
        .sort_values("BILL_ID", ascending=False)
        .reset_index()
        .rename(columns={"BILL_ID": "BILL_COUNT"})
        .head(3)
    )
    print("\n")

bills.groupby(["COMMITTEE_NAME", "BILL_NAME"]).count()[["BILL_ID"]].sort_values("BILL_ID", ascending=False).reset_index().rename(
    columns={
        "BILL_ID": "BILL_COUNT",
    }
)

In [ ]:
bills[(bills["STATUS"]=="원안가결") | (bills["STATUS"]=="수정가결")].groupby(["COMMITTEE_NAME", "BILL_NAME"]).count()["BILL_ID"].reset_index().rename(
    columns={
        "BILL_ID": "BILL_COUNT",
    }
).sort_values("BILL_COUNT", ascending=False)

## 의원

In [ ]:
member_ratio = members.groupby("POLY_NM").count()["MONA_CD"].reset_index().rename(
    columns={
        "MONA_CD": "MEMBER_COUNT",
    }
).sort_values("MEMBER_COUNT", ascending=False)
member_ratio["MEMBER_RATIO"] = (
    member_ratio["MEMBER_COUNT"] / member_ratio["MEMBER_COUNT"].sum()
).round(2)
member_ratio.sort_values("MEMBER_RATIO", ascending=False)

In [ ]:
from collections import defaultdict
poly = defaultdict(int)

bp = proposers.merge(
    members[["MONA_CD", "POLY_NM"]],
    left_on="PROPOSER_ID",
    right_on="MONA_CD",
    how="inner",
).drop(columns=["MONA_CD"]).groupby("BILL_ID").agg(
    {
        "POLY_NM": lambda x: dict(x.value_counts()),
        "PROPOSER_ID": lambda x: list(set(x)),
        "PROPOSER_ID": lambda x: ", ".join(x),
    }
).reset_index().rename(
    columns={
        "POLY_NM": "PROPOSER_POLY_NM",
        "PROPOSER_ID": "PROPOSER_IDS",
    }
)
bp = bp[bp['PROPOSER_POLY_NM'].apply(lambda x: len(x) > 1)]
ana = bills[bills['BILL_ID'].isin(bp['BILL_ID'])]

In [ ]:
ana.groupby(["COMMITTEE_NAME", "BILL_NAME"]).count()["BILL_ID"].reset_index().rename(
    columns={
        "BILL_ID": "BILL_COUNT",
    }
).sort_values("BILL_COUNT", ascending=False)

In [ ]:
bill_props = proposers.merge(
    members[["MONA_CD", "POLY_NM"]],
    left_on="PROPOSER_ID",
    right_on="MONA_CD",
    how="inner",
).drop(columns=["MONA_CD"]).merge(
    bills[["BILL_ID", "COMMITTEE_NAME", "BILL_NAME"]],
    on="BILL_ID",
    how="inner",
)

In [ ]:
bill_props

## 정당별

In [ ]:
lead_props = bill_props[bill_props["PROPOSER_TYPE"] == "의원대표"]
# co_props = bill_props[bill_props["PROPOSER_TYPE"] == "의원공동"]

In [ ]:
data = lead_props.groupby(["POLY_NM"]).count()["BILL_ID"].reset_index().rename(
    columns={
        "BILL_ID": "BILL_COUNT",
    }
).sort_values("BILL_COUNT", ascending=False)
data = data.merge(member_ratio[["POLY_NM", "MEMBER_COUNT"]], on="POLY_NM", how="left")
data["RATIO"] = data["BILL_COUNT"] / data["MEMBER_COUNT"]
data.sort_values("RATIO", ascending=False)
data

In [ ]:
data = lead_props.groupby(["POLY_NM"]).count()["BILL_ID"].reset_index().rename(
    columns={
        "BILL_ID": "BILL_COUNT",
    }
).sort_values("BILL_COUNT", ascending=False)
data = data.merge(member_ratio, on="POLY_NM", how="left")
data["RATIO"] = data["BILL_COUNT"] / data["MEMBER_COUNT"]
data.sort_values("RATIO", ascending=False)
data

In [ ]:
poly_comm = lead_props.groupby(["POLY_NM", "COMMITTEE_NAME"]).count()["BILL_ID"].reset_index().rename(
    columns={
        "BILL_ID": "BILL_COUNT",
    }
).sort_values("BILL_COUNT", ascending=False)
poly_comm = poly_comm.merge(member_ratio, on="POLY_NM", how="left")
poly_comm["RATIO"] = poly_comm["BILL_COUNT"] / poly_comm["MEMBER_COUNT"]

In [ ]:
poly_comm[poly_comm["POLY_NM"] == "더불어민주당"].sort_values("RATIO", ascending=False)

In [ ]:
poly_comm[poly_comm["POLY_NM"] == "국민의힘"].sort_values("RATIO", ascending=False)

In [ ]:
poly_comm[poly_comm["POLY_NM"] == "조국혁신당"].sort_values("RATIO", ascending=False)

In [ ]:
poly_comm[poly_comm["POLY_NM"] == "진보당"].sort_values("RATIO", ascending=False)

In [ ]:
poly_bill = lead_props.groupby(["POLY_NM", "BILL_NAME"]).count()["BILL_ID"].reset_index().rename(
    columns={
        "BILL_ID": "BILL_COUNT",
    }
).sort_values("BILL_COUNT", ascending=False)
poly_bill = poly_bill.merge(member_ratio[["POLY_NM", "MEMBER_COUNT"]], on="POLY_NM", how="left")

poly_bill["RATIO"] = poly_bill["BILL_COUNT"] / poly_bill["MEMBER_COUNT"]

In [ ]:
poly_bill[poly_bill["POLY_NM"] == "더불어민주당"].sort_values("RATIO", ascending=False)

In [ ]:
poly_bill[poly_bill["POLY_NM"] == "국민의힘"].sort_values("RATIO", ascending=False)

In [ ]:
poly_bill[poly_bill["POLY_NM"] == "조국혁신당"].sort_values("RATIO", ascending=False)

In [ ]:
poly_bill[poly_bill["POLY_NM"] == "진보당"].sort_values("RATIO", ascending=False)